In [5]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 163 kB in 3s (55.8 kB/s)
Reading package lists... Don

In [6]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-06-11 15:25:17--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.52MB/s    in 0.6s    

2022-06-11 15:25:19 (1.52 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [8]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

## Drop duplicates and incomplete rows

In [9]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

2643619
2643241
2643241


## Examine the schema

In [10]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Rename columns

In [11]:
# df1 = df.withColumnRenamed("Employee ID", "employee_id") \
#         .withColumnRenamed("Email", "email") \
#         .withColumnRenamed("Gender", "gender") \
#         .withColumnRenamed("Hire Date", "hire_date") \
#         .withColumnRenamed("DOB", "dob") \
#         .withColumnRenamed("Encrypted Password", "password")
# df1.show(5)
# NOTE: The above code from Homework Lesson 22.3.4 was not needed since all of the columns aligned to what was needed:)
df1=df
df1

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

## Create a new DataFrame for products 

In [12]:
products = df1.select(["product_id", "product_title"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00BUQ4AV8|Evergreens Pet Do...|
|B00RVY9DRO|Volkman Avian Sci...|
|B004M8LM44|Dogtra YS300 Dog ...|
|B0006IZH68|thedog8it! Gourme...|
|B0014N70QO|Litter Kwitter To...|
+----------+--------------------+
only showing top 5 rows



## Write DataFrame to RDS

In [13]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://rdsdb.cgx5vst360bw.us-east-2.rds.amazonaws.com:5432/dbforbigdata"
print("Need to change password below as per BZ Gmail email on 6/8/22 at 10:50pm")
print("change it from xxxxxxxx to temporary one 1 per BZ Gmail email on 6/8/22 at 10:50pm")
config = {"user":"postgres",
          "password": "xxxxxxxx",
          "driver":"org.postgresql.Driver"}

Need to change password below as per BZ Gmail email on 6/8/22 at 10:50pm
change it from xxxxxxxx to temporary one 1 per BZ Gmail email on 6/8/22 at 10:50pm


In [14]:
# Write DataFrame to table

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

## Create a new DataFrame for review_id_table

In [15]:
review_id_table = df1.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100EO2AQP6B02|   33140435|B00BUQ4AV8|     805940477| 2014-10-12|
|R100IITU077KKZ|   25475634|B00RVY9DRO|     189955978| 2015-06-04|
|R100WIAVSCW4D1|   49229898|B004M8LM44|     419962202| 2013-07-31|
|R1023NXBAJ0BYT|   52227830|B0006IZH68|     470609741| 2004-12-23|
|R102CEVVIBXNFB|   11062585|B0014N70QO|     226028756| 2014-07-14|
|R102MO11LN79HV|   17155412|B0039LFVUG|     517238077| 2010-07-24|
|R102TPQOB6ECI4|   46107297|B008QVKBLS|     212972268| 2015-02-27|
|R1032IWOCCKRV1|   28307907|B00406YHRC|     178991753| 2014-01-08|
|R103QNPJX6OJ6D|   38188681|B003TUQWB4|     287424702| 2014-05-27|
|R104GDOC82SGT4|   49476810|B000YIWUXI|     178342731| 2015-03-13|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



## Write DataFrame to RDS





In [16]:
# Write DataFrame to review_id_table

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

## Create a new DataFrame for customers:

In [17]:
# Since customer_count isn't available in the Amazon supplied Datasets, first need to count customer occurences of customer_id, then rename columns, then 
customers = df1.select(["customer_id"])
customers.show(10)

customers = customers.groupBy("customer_id").count()
customers.show(10)

customers.orderBy("customer_id").select(["customer_id", "count"])
customers.show(10)

customers = customers.withColumnRenamed("count","customer_count")
customers.show(10)

customers.printSchema()

+-----------+
|customer_id|
+-----------+
|   33140435|
|   25475634|
|   49229898|
|   52227830|
|   11062585|
|   17155412|
|   46107297|
|   28307907|
|   38188681|
|   49476810|
+-----------+
only showing top 10 rows

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   37223012|    5|
|   52055901|    1|
|   16223525|    5|
|   28023775|    3|
|   51092659|    3|
|   44748307|   11|
|   33283869|    1|
|   14670648|   21|
|   21340218|    1|
|   40656071|    2|
+-----------+-----+
only showing top 10 rows

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   37223012|    5|
|   52055901|    1|
|   16223525|    5|
|   28023775|    3|
|   51092659|    3|
|   44748307|   11|
|   33283869|    1|
|   14670648|   21|
|   21340218|    1|
|   40656071|    2|
+-----------+-----+
only showing top 10 rows

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   37223012|             5|
|   52055901|             1|
|   16223525|        

## Write DataFrame to RDS

In [18]:
# Write DataFrame to table
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

## Create a new DataFrame for vine_table

---



In [19]:
vine_table = df1.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100EO2AQP6B02|          3|            2|          4|   N|
|R100IITU077KKZ|          5|            0|          0|   N|
|R100WIAVSCW4D1|          5|            3|          5|   N|
|R1023NXBAJ0BYT|          5|            0|          0|   N|
|R102CEVVIBXNFB|          1|            0|          2|   N|
|R102MO11LN79HV|          5|            0|          0|   N|
|R102TPQOB6ECI4|          3|            0|          0|   N|
|R1032IWOCCKRV1|          1|            2|          2|   N|
|R103QNPJX6OJ6D|          5|            1|          1|   N|
|R104GDOC82SGT4|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



## Write DataFrame to RDS

In [20]:
# Write DataFrame to table

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

## Printing out the counts for each table 
### (Will use it to compare the counts 
## via SQL querries)


In [21]:
print(products.count())
print(review_id_table.count())
print(customers.count())
print(vine_table.count())


2643241
2643241
1414974
2643241
